In [22]:
# 데이터 프레임
import pandas as pd
import numpy as np

In [23]:
# 텍스트 분석
from konlpy.tag import Okt
import nltk

In [24]:
# 머신러닝, 딥러닝
from sklearn.model_selection import train_test_split

from keras.models import Sequential, load_model
from keras.layers import Dense 
from keras import optimizers, losses, metrics

In [25]:
# 기타
import os, csv

In [26]:
def read_csv(path_list):
    return pd.concat([pd.read_csv("data/" + path + ".csv", encoding="utf-8-sig") for path in path_list])\
            .reset_index().drop('index', axis=1)

In [27]:
def pos_tagging(data, fname): # pos = Part Of Speech (형태소)
    okt = Okt()
    data['comment'] = data['comment'].apply(lambda x: ['/'.join(t) for t in okt.pos(x, norm=True, stem=True)])
    data.to_csv(fname, index=False)
    return data

In [28]:
def pos_check(fname):
    return pd.read_csv(fname) if os.path.isfile(fname)\
    else pos_tagging(read_csv(["pos_opinion12", "neg_opinion12", "pos_opinion19", "neg_opinion19"]), fname)

In [50]:
def word_selecting(column, num=None):
    tokens = [token for row in column.tolist() for token in row]
    text = nltk.Text(tokens, name='NMSC')
    selected_words = [t[0] for t in text.vocab().most_common(10000)]
    pd.DataFrame(selected_words).to_csv("selected_words.csv", header=None)
    return selected_words

In [45]:
def convert_to_float(data):
    fname = "selected_words.csv"
    selected_words = pd.read_csv(fname, header=None) if os.path.isfile(fname) else word_selecting(data['comment'])
#     def term_frequency(doc, selected_words):
#         return [doc.count(word) for word in selected_words]
#     t_freq = [term_frequency(d, selected_words) for d in list(data['comment'])]
#     return {"data" : np.asarray(t_freq).astype('float32'), "label": df['risk'].astype('float32')}

In [39]:
data = pos_check("pos_tag.csv")

In [60]:
data.iloc[0,1]

"['잘/VerbPrefix', '되다/Verb', '다운로드/Noun']"

In [56]:
[token for row in data['comment'] for token in row]

['[',
 "'",
 '잘',
 '/',
 'V',
 'e',
 'r',
 'b',
 'P',
 'r',
 'e',
 'f',
 'i',
 'x',
 "'",
 ',',
 ' ',
 "'",
 '되',
 '다',
 '/',
 'V',
 'e',
 'r',
 'b',
 "'",
 ',',
 ' ',
 "'",
 '다',
 '운',
 '로',
 '드',
 '/',
 'N',
 'o',
 'u',
 'n',
 "'",
 ']',
 '[',
 "'",
 '제',
 '/',
 'N',
 'o',
 'u',
 'n',
 "'",
 ',',
 ' ',
 "'",
 '가',
 '하',
 '다',
 '/',
 'V',
 'e',
 'r',
 'b',
 "'",
 ',',
 ' ',
 "'",
 'r',
 'p',
 'g',
 '/',
 'A',
 'l',
 'p',
 'h',
 'a',
 "'",
 ',',
 ' ',
 "'",
 '게',
 '임',
 '/',
 'N',
 'o',
 'u',
 'n',
 "'",
 ',',
 ' ',
 "'",
 '에',
 '서',
 '가',
 '/',
 'J',
 'o',
 's',
 'a',
 "'",
 ',',
 ' ',
 "'",
 '장',
 '/',
 'N',
 'o',
 'u',
 'n',
 "'",
 ',',
 ' ',
 "'",
 '재',
 '미',
 '있',
 '다',
 '/',
 'A',
 'd',
 'j',
 'e',
 'c',
 't',
 'i',
 'v',
 'e',
 "'",
 ',',
 ' ',
 "'",
 '게',
 '임',
 '/',
 'N',
 'o',
 'u',
 'n',
 "'",
 ',',
 ' ',
 "'",
 '이',
 '다',
 '/',
 'A',
 'd',
 'j',
 'e',
 'c',
 't',
 'i',
 'v',
 'e',
 "'",
 ']',
 '[',
 "'",
 '제',
 '발',
 '/',
 'N',
 'o',
 'u',
 'n',
 "'",
 ',',
 ' ',
 "'",
 '사'

In [46]:
dataset = convert_to_float(data)

In [10]:
def deep_learning(dataset):
    train_data, test_data, train_label, test_label = train_test_split(dataset['data'], dataset['label'])
    
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

    model.fit(train_data, train_label, epochs=100, batch_size=512)
    model.save('model.h5')
    print(model.evaluate(test_data, test_label))
    return model
def read_model(fname):
    return load_model(fname) if os.path.isfile(fname) else deep_learning(preprocessing("pos_tag.csv"))

In [11]:
read_model("model.h5")

TypeError: must be str, not int